In [4]:
from src.distances import dmat_and_order
import numpy as np
from scipy.sparse.csgraph import dijkstra
import networkx as nx
from ipywidgets import interact

In [5]:
from tqdm.notebook import tqdm

In [12]:
def spanner_adj(distances, t):
    ind = np.unravel_index(np.argsort(distances, axis=None), distances.shape)
    adj = np.zeros(distances.shape)
    total = 1.0*distances.shape[0] * distances.shape[1]
    for i,j in tqdm(zip(ind[0], ind[1]), total=len(ind[0])):
        if i<j: continue
        d = distances[i,j]
        shortest = dijkstra(adj, indices=i, directed=False, min_only=True, limit=t*d)[j]
        if shortest > t*d:
            adj[i,j] = d
            adj[j,i] = d
    return adj

In [17]:
adj = spanner_adj(dmat_sym, 1.5)

  0%|          | 0/279841 [00:00<?, ?it/s]

In [18]:
np.count_nonzero(adj)/np.count_nonzero(dmat_sym)

0.04234690954917798

In [7]:
dmat, dmat_sym, order = dmat_and_order("vega-lite", "vega-lite", "nmi")

@interact
def do(t=(1.0,5.0)):
    return nx.draw_kamada_kawai(nx.from_numpy_matrix(np.tril(spanner_adj(dmat_sym, t))))

interactive(children=(FloatSlider(value=3.0, description='t', max=5.0, min=1.0), Output()), _dom_classes=('wid…